In [58]:
import pandas as pd
import numpy as np

In [59]:
# Frame containing all the useful values

fundings = pd.read_excel ('grant.xlsx') [['University', 'Approved Amount']]
fundings.head ()

,University,Approved Amount
0,Nicht zuteilbar - NA,11619.00
1,Université de Genève - GE,41022.00
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00
3,Universität Basel - BS,52627.00
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00


In [9]:
# Cantons keys

cantons = pd.read_csv ('cantons.csv')
cantons.head ()

,Canton
0,ZH
1,BE
2,LU
3,UR
4,SZ


In [66]:
# List of all the universities

universities = pd.DataFrame(fundings ['University'].unique ())

In [99]:
universities = universities[1:]
universities['Canton'] = universities['University'].apply(get_canton)
universities

C:\Users\Fearnley\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,University,Canton
1,Université de Genève - GE,GE
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",None
3,Universität Basel - BS,BS
4,Université de Fribourg - FR,FR
5,Universität Zürich - ZH,ZH
6,Université de Lausanne - LA,VD
7,Universität Bern - BE,BE
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",None
9,Université de Neuchâtel - NE,NE
10,ETH Zürich - ETHZ,ZH


In [101]:
universities.to_csv('uni_canton_mapping.csv')

In [124]:
def get_canton(university):
    from credentials import places_api_key
    # Get placeid from university name
    try:
        url_text_search = 'https://maps.googleapis.com/maps/api/place/textsearch/json?key={key}&query={query}'.format(
                key=places_api_key,
                query=university
        )
        r=requests.get(url_text_search)
        print(r.json())
        if len(r.json()['results']) > 0:
            placeid = r.json()['results'][0]['place_id']
            print(placeid)
            # Get info on placeid
            url_place_search = 'https://maps.googleapis.com/maps/api/place/details/json?key={key}&placeid={placeid}'.format(
                key=places_api_key,
                placeid=placeid
            )
            r2=requests.get(url_place_search)
            print(r2.json())
            # Parse json and return canton
            for el in r2.json()['result']['address_components']:
                if el['types']==['administrative_area_level_1', 'political']:
                    return el['short_name']
        else:
            return None
    except Exception as e:
        print(e)
        return None

In [125]:
get_canton('Biotechnologie Institut Thurgau - BITG')

{'status': 'OK', 'results': [{'types': ['clothing_store', 'jewelry_store', 'furniture_store', 'home_goods_store', 'store', 'point_of_interest', 'establishment'], 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'Biotechnologie Institut Thurgau (BITg)', 'formatted_address': 'Unterseestrasse 47, 8280 Kreuzlingen, Switzerland', 'place_id': 'ChIJj4vWuNv2mkcRfOGoYejYRVw', 'id': 'd31367348596c478587fb251dc432d9884da7ad1', 'reference': 'CmRRAAAA1-bTeDFy9e4aVcI-1cq8hlFIAODpdFyqdzU0IFUPkSevMD0cn6zF1gj_2dMSnnrZSfC55QIEQj6abIfw6EFSpQcgAVvsDOPfUKH4Kd7sOb6Bemp02wHDD6kg2gmY2YyPEhDhXVndPJ7mME0ZJZgkGYB7GhTUmuZCRb5foADzSwGS0kqz9jp06Q', 'geometry': {'location': {'lat': 47.64832730000001, 'lng': 9.161303}, 'viewport': {'northeast': {'lat': 47.64847415, 'lng': 9.161376649999998}, 'southwest': {'lat': 47.64827835000001, 'lng': 9.161278450000001}}}}], 'html_attributions': []}
ChIJj4vWuNv2mkcRfOGoYejYRVw
{'status': 'OVER_QUERY_LIMIT', 'error_message': 'You have exceeded yo

# Tasks left for the homework : 
### - classify each university to its canton, then groupyby canton,
### - sum the approved amount into a column named 'Granted Amount'
### - call the function 'buildChoroplethMap' from the script 'choropleth' to produce the map. The final dataFrame should have two columns 'Canton' and 'Granted Money'.

# Example of use for function 'buildChoroplethMap'

In [13]:
testFrame = pd.merge (cantons, pd.DataFrame (np.random.randint(0,100,size=(100, 1)), columns = ['Granted Money']), left_index = True, right_index = True)
testFrame.head ()

,Canton,Granted Money
0,ZH,22
1,BE,88
2,LU,97
3,UR,52
4,SZ,53


## Parameters :

 - dataFrame : the data frame containing canton values
 - scale : color scales on the map
 - outputFileName : the name of the output map file